In [33]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [47]:
image_path = 'media/Cat.jpg'
LAYER_NAME = 'block5_conv3'
CAT_CLASS_INDEX = 281

In [48]:
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)

In [49]:
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

In [50]:
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])


In [51]:
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CAT_CLASS_INDEX]

In [52]:
conv_outputs.shape

TensorShape([1, 14, 14, 512])

In [53]:
# Extract filters and gradients
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

In [54]:
# Guided grad
gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

In [55]:
# Average gradients spatially
weights = tf.reduce_mean(guided_grads, axis=(0, 1))

In [56]:
# Build a ponderated map of filters according to gradients importance
cam = np.ones(output.shape[0:2], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, index]

# Heatmap visualization
cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)


In [57]:
cv2.imshow('image',output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()